# Kafka Topic Management

Topics in Kafka are typically auto created. In this notebook we can explore the topics that already exists on the server and, optionally, we can change the replication factor or number of partitions for a topic

In [15]:
from confluent_kafka.admin import AdminClient

# Kafka broker addresses
KAFKA_BROKER = 'broker:29092,broker-2:29092'

# Initialize the AdminClient
admin_client = AdminClient({'bootstrap.servers': KAFKA_BROKER})

# Retrieve cluster metadata
cluster_metadata = admin_client.list_topics(timeout=10)

print("Topic List with Partition and Replication Details:\n")

for topic_name, topic_metadata in cluster_metadata.topics.items():
    # Skip internal topics (those that start with '__')
    if topic_name.startswith('__'):
        continue

    partition_count = len(topic_metadata.partitions)
    # Assume replication factor is the number of replicas of the first partition
    # Check if there are partitions
    if partition_count > 0:
        first_partition = next(iter(topic_metadata.partitions.values()))
        replication_factor = len(first_partition.replicas)
    else:
        replication_factor = 'Unknown'  # Topic with no partitions

    print(f"Topic '{topic_name}': Partitions = {partition_count}, Replication Factor = {replication_factor}")



Topic List with Partition and Replication Details:

Topic 'docker-connect-status': Partitions = 5, Replication Factor = 2
Topic 'iot_data': Partitions = 1, Replication Factor = 1
Topic 'docker-connect-configs': Partitions = 1, Replication Factor = 2
Topic '_schemas': Partitions = 1, Replication Factor = 2
Topic 'docker-connect-offsets': Partitions = 25, Replication Factor = 2
Topic 'github_events': Partitions = 1, Replication Factor = 1
Topic 'trades': Partitions = 1, Replication Factor = 2
Topic '_confluent-monitoring': Partitions = 1, Replication Factor = 1
Topic 'smart-meters': Partitions = 1, Replication Factor = 1


In [14]:
from confluent_kafka.admin import AdminClient, NewTopic
import time

KAFKA_BROKER = 'broker:29092,broker-2:29092'
KAFKA_TOPIC = 'trades'
PARTITIONS = 1
REPLICATION_FACTOR = 2  # Should not exceed the number of brokers in your cluster

# Initialize the AdminClient
admin_client = AdminClient({'bootstrap.servers': KAFKA_BROKER})

# Check if the topic exists
metadata = admin_client.list_topics(timeout=10)
if KAFKA_TOPIC in metadata.topics:
    # Delete the topic if it exists
    delete_futures = admin_client.delete_topics([KAFKA_TOPIC])
    for topic, future in delete_futures.items():
        try:
            future.result()  # Wait for the operation to finish
            print(f"Topic '{topic}' deleted.")
        except Exception as e:
            print(f"Failed to delete topic '{topic}': {e}")
            # You may choose to exit or handle the error as needed

    # Wait until the topic is fully deleted    
    while True:
        metadata = admin_client.list_topics(timeout=10)
        if KAFKA_TOPIC not in metadata.topics:
            print(f"Topic '{KAFKA_TOPIC}' has been fully deleted.")
            break
        else:
            print(f"Waiting for topic '{KAFKA_TOPIC}' to be deleted...")
            time.sleep(1)

# Create a new topic with the desired number of partitions
new_topic = NewTopic(
    topic=KAFKA_TOPIC,
    num_partitions=PARTITIONS,
    replication_factor=REPLICATION_FACTOR
)

create_futures = admin_client.create_topics([new_topic])
for topic, future in create_futures.items():
    try:
        future.result()  # Wait for the operation to finish
        print(f"Topic '{topic}' created with {PARTITIONS} partitions.")
    except Exception as e:
        print(f"Failed to create topic '{topic}': {e}")
        # You may choose to exit or handle the error as needed

Topic 'trades' deleted.
Topic 'trades' has been fully deleted.
Topic 'trades' created with 1 partitions.
